In [1]:
# read all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import interact, widgets
import csv

In [2]:
# load data from ETP_data.csv

data = pd.read_csv('ETP_data.csv', index_col='datum', parse_dates=True)
ETP = data.ETP
Re = data.Re

In [3]:
# 

def linear_reduction_function(S_B_A, available_water_capacity, fraction_of_awc=0.7):
    """
    This function calculates the water stress coefficient K_s based on the volumetric soil moisture.
    Parameters
    ----------
    S_B_A : float
        The volumetric soil moisture in %
    available_water_capacity : float
        The volumetric soil moisture at the field capacity in %
    fraction_of_awc : float
        The fraction of the field capacity at which the plant starts to experience water stress (awc =available_water_capacity)
    Returns
    -------
    f_r : float
        The water stress coefficient K_s
    """ 

    # calculate the "tipping point"
    theta_0 = available_water_capacity * fraction_of_awc

    if S_B_A >= theta_0:
        f_r = 1
    #volumetric soil moisture-> S_B_A 

    elif S_B_A <= 0:
        f_r = 0

    elif S_B_A > 0 and S_B_A < theta_0:
        f_r = (S_B_A ) / (theta_0 )

    return f_r


In [4]:
def update_moisture_content(soilwater_content, PP, ETR, awc):
    #awc = actual/available water capacity???
    #PP = precipitation???
    
    # check if water holding capacity is exceeded, and if it is: update the awc and calculate the excess water...
    if soilwater_content + PP - ETR > awc:
        updated_value = awc
        excess_water = soilwater_content + PP - ETR - awc
        
    #...else set the exsess water: 0
    else: 
        updated_value = soilwater_content + PP - ETR
        excess_water = 0

    return updated_value, excess_water


In [5]:
def run_simulation(awc=150, p=0.7, xrain=1):
    Re_mm_day = Re / 2.45
    # Re...extraterrestrial radiation [MJ/(mm2 d)]
    # Extraterrestrial radiation [mm/d]
    
    # set all initial variables
    Precip = data['N (mm/d)'] * xrain

    gw_flux = np.zeros(len(data))

    soil_moisture = np.zeros(len(data))
    initial_soil_moisture = awc
    soil_moisture[0] = initial_soil_moisture

    initial_fraction_of_ETP = 1
    ETR = np.zeros(len(data))
    
    # now update them
    for i in np.arange(1, len(data)):
        
        etp = ETP[i]
        ETR[0] = initial_fraction_of_ETP * etp
        
        factor = linear_reduction_function(soil_moisture[i-1], awc, p)
        ETR[i] = factor * ETP[i]

        soil_moisture[i], excess_water = update_moisture_content(soil_moisture[i-1], Precip[i], ETR[i], awc)

        gw_flux[i] = excess_water

    return Precip, ETP, ETR, soil_moisture, gw_flux


In [6]:
#plotting the data
def plot_simulation(awc=150, p=0.7, xrain=1):
    
    # run the simulation to create data for Precip, ETP, ETR, soil_moisture and gw_flux, 
    # by handing values for awc, p and xrain
    Precip, ETP, ETR, soil_moisture, gw_flux = run_simulation(awc, p, xrain)

    # define the size and start plotting
    fig = plt.figure(figsize=(10, 7))

    # Define the width ratio and spacing
    width_ratio = 0.3
    spacing_vertical = 0.05
    spacing_horizontal = 0.09

    # Calculate left plot width and right plot width considering the spacing
    left_plot_width = (1 - width_ratio - spacing_horizontal)
    right_plot_width = width_ratio

    # Heights and vertical positions for left plots
    height_left = (1 - 4 * spacing_vertical) / 3
    positions_left = [2/3 + spacing_vertical, 1/3 + spacing_vertical, 0 + spacing_vertical]

    # Create the existing subplots
    ax1 = fig.add_axes([0, positions_left[0], left_plot_width, height_left])
    ax2 = fig.add_axes([0, positions_left[1], left_plot_width, height_left])
    ax3 = fig.add_axes([0, positions_left[2], left_plot_width, height_left])

    # Heights and vertical positions for right plots
    height_right = (1 - 2 * spacing_vertical) / 2
    positions_right = [1/2 + spacing_vertical/2, 0 + spacing_vertical-0.05]

    # Create the new subplots on the right
    ax_right1 = fig.add_axes([1 - right_plot_width, positions_right[0], right_plot_width, height_right])
    ax_right2 = fig.add_axes([1 - right_plot_width, positions_right[1], right_plot_width, height_right])  

    for ax in [ax_right1, ax_right2]:
        ax.yaxis.tick_right()
        ax.yaxis.set_label_position("right")

    # Now plot your data as before
    ## plot plot precipitation
    ax1.bar(data.index, Precip.values, color='skyblue', label='precipitation', width=1)
    ax1.set_ylim(max(Precip), 0)  # This inverts the y-axis

    ## plot the ETP
    ax2.plot(data.index, ETP, color='goldenrod', label='ETP')

    ## plot the ETR
    ax2.plot(data.index, ETR, color='tomato', label='ETR')

    ## plot the soil moisture storage
    ax3.plot(data.index, soil_moisture, color='steelblue', label='soil moisture storage')

    ax4 = ax3.twinx()
    ## plot the groundwater flux
    ax4.bar(data.index, gw_flux, color='navy', label='groundwater flux', width=1)
    S_B_A = np.arange(0, 150, 1)
    curve = np.zeros(len(S_B_A))

    # Cumulative distribution of precipitation and gw_flux
    ##creating cumulativ data
    cumulative_precip = np.cumsum(Precip.values)
    cumulative_gw_flux = np.cumsum(gw_flux)
    ##plot
    ax_right1.plot(data.index, cumulative_precip, color='skyblue', label='cumulative precipitation')
    ax_right1.plot(data.index, cumulative_gw_flux, color='navy', label='cumulative groundwater flux')

    # Cumulative distribution of ETP and ETR
    ##creating cumulativ data
    cumulative_ETP = np.cumsum(ETP)
    cumulative_ETR = np.cumsum(ETR)
    ##plot
    ax_right2.plot(data.index, cumulative_ETP, color='goldenrod', label='cumulative ETP')
    ax_right2.plot(data.index, cumulative_ETR, color='tomato', label='cumulative ETR')
 
   
    # add labels and legend
    ##left
    ax1.set_ylabel('precipitation (mm/d)')
    ax2.set_ylabel('ETP and ETR (mm/d)')
    ax3.set_ylabel('soil moisture (mm)')
    ax4.set_ylabel('groundwater flux (mm/d)')
    ax1.legend(loc='best')
    ax2.legend(loc='best')
    ax3.legend(loc='upper left')
    ax4.legend(loc='center right')
    ##right
    ax_right1.set_ylabel('precipitation and gw_flux (mm/d)')
    ax_right2.set_ylabel('ETP and ETR (mm/d)') 
    ax_right1.legend(loc='upper left')
    ax_right2.legend(loc='best') 
    ## interval for axes
    ax_right1.xaxis.set_major_locator(ticker.MultipleLocator(120))
    ax_right2.xaxis.set_major_locator(ticker.MultipleLocator(120))

In [7]:
# creating sliders for the imput values
interact(plot_simulation, 
         fc=widgets.FloatSlider(min=1, max=500, step=5, value=150),
         p=widgets.FloatSlider(min=0.5, max=1.0, step=0.1, value=0.7),
         xrain=widgets.FloatSlider(min=0.1, max=5, step=0.5, value=1))

interactive(children=(IntSlider(value=150, description='awc', max=450, min=-150), FloatSlider(value=0.7, descr…

<function __main__.plot_simulation(awc=150, p=0.7, xrain=1)>